In [2]:
from functools import partial
from datetime import datetime
# from pytz import timezone
from models_vgg16 import model_build, IntervalEval, compute_metrics, model_build_finetune
from utils_vgg16 import parse_example
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import matplotlib.pyplot as plt
import shutil
import os
os.environ['TF_CUDNN_DETERMINISTIC']='1'

In [3]:
# Do not change these params
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 32
IMAGE_SIZE = (227, 227, 3)
TEST_SIZE = 0.1 

# Directory and Model name
MODEL_NAME = 'vgg16' #@param ["densenet121", "resnet50", "vgg16"]{type:"raw"}
DATA_DIR = 'tfrecords'
MODEL_DIR = 'models'
TB_DIR = 'tensorboard'

## Parameters for Turning ##
LR = 0.0001 #@param {type:"slider", min:1e-4, max:1e-3, step:1e-4}
NUM_EPOCHS = 6 #@param {type:"slider", min:5, max:15, step:1}
DROP_OUT = 0.2 #@param {type:"slider", min:0, max:0.5, step:0.05}
OHEM_RATE = 0.5 #@param {type:"slider", min:0.1, max:0.8, step:0.1}
DATA_AUG = True #@param {type:"boolean"}
CV_MODE = False #@param {type:"boolean"}
config_num = 15

# set tensorflow global random seed
tf.random.set_seed(1234)

In [4]:
all_tfrecords = [os.path.join(DATA_DIR, x) for x in os.listdir(DATA_DIR)]
train_val, test = train_test_split(all_tfrecords, test_size=TEST_SIZE, random_state=SEED)

In [5]:
# test data setup (DO NOT MODIFY)
test_dataset = tf.data.TFRecordDataset(np.asarray(test))
test_dataset = test_dataset.map(partial(parse_example, 
                                            data_aug = False,
                                            shape=IMAGE_SIZE)).prefetch(AUTOTUNE).batch(BATCH_SIZE)
test_samples = sum(int(x.split('.')[0].split('_')[1]) for x in np.asarray(test))

test_labels = np.zeros((test_samples, ), dtype=np.uint8)
for i, (_, label) in enumerate(test_dataset):
    test_labels[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = label

In [10]:
# Initiate model
model = model_build(1000,
                    drop_out=DROP_OUT,
                    lr=LR, 
                    ohem_rate=OHEM_RATE, 
                    num_epochs=NUM_EPOCHS, 
                    model_name=MODEL_NAME)

In [11]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 227, 227, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 227, 227, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 113, 113, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 113, 113, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 113, 113, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [12]:
model.load_weights('./vgg16_fold0_val80190_test79251_FineTuningWoLastLayerPreTuning.h5')

In [13]:
test_preds = model.predict(test_dataset, verbose=0)
test_auc, test_pre, test_rec, test_f1 = compute_metrics(test_labels, test_preds)
pd.DataFrame(test_preds, columns=['probs']).to_csv('vgg16_val80190_test79251.csv', index=False)

In [14]:
test_f1

0.7925117004680187

In [15]:
np.save('vgg16_val80190_test79251.npy', test_preds)

In [16]:
data = np.load('vgg16_val80190_test79251.npy')

In [17]:
data

array([[0.89537966],
       [0.95278233],
       [0.87801003],
       [0.68222904],
       [0.5017303 ],
       [0.7260857 ],
       [0.9341215 ],
       [0.10986327],
       [0.22361848],
       [0.8749583 ],
       [0.912667  ],
       [0.87857574],
       [0.14242068],
       [0.74869746],
       [0.95639396],
       [0.5879456 ],
       [0.18952137],
       [0.21738642],
       [0.06298125],
       [0.5617227 ],
       [0.13455524],
       [0.04503578],
       [0.3474366 ],
       [0.3976055 ],
       [0.06757977],
       [0.29002956],
       [0.92019755],
       [0.29481795],
       [0.28220698],
       [0.3918963 ],
       [0.35234705],
       [0.05669105],
       [0.77217585],
       [0.8378763 ],
       [0.8965531 ],
       [0.16397732],
       [0.825985  ],
       [0.76335883],
       [0.95562065],
       [0.52908397],
       [0.88950247],
       [0.9226084 ],
       [0.91299826],
       [0.9277266 ],
       [0.8021072 ],
       [0.84574115],
       [0.8736719 ],
       [0.291